In [ ]:
from xmm.preprocess._expr2ast import expr2ast
from xmm.preprocess._ast2sympy import ast2sympy


def main(expression):
    # Parse the expression into an AST
    ast_tree = expr2ast(expression)

    # Create SymPy expression from the AST
    sympy_expr = ast2sympy(ast_tree)

    print("SymPy expression:", sympy_expr)

    return sympy_expr

# Example usage
expression = "c1 * (c2 * c2 * (r1 + c3) * (r1 + c3) - 1) * exp(-0.5 * (r1 * r1 * c2 * c2))"
expr = main(expression)
expr

In [ ]:
from xmm.preprocess._ast2CUDAexpr import ast2CUDAexpr

def main_taichi(expression):
    # Parse the expression into an AST
    ast_tree = expr2ast(expression)

    # Convert the AST directly to a Taichi expression string
    taichi_expr = ast2CUDAexpr(ast_tree.body)

    print("CUDA expression:", taichi_expr)

    return taichi_expr

# Example usage
expression = "log(c1 * (c2 * c2 * (r1 + c3)) * atan2(r1 + c3, c2) - 1) * exp(-0.5 * (r1 * r1 * c2 * c2))"
taichi_expr = main_taichi(expression)

In [ ]:
from xmm.preprocess._sympy2ast import sympy2ast
import astor

# Replacing the main function to include conversion back to AST
def around(expression):
    # Parse the expression into an AST
    ast_tree = expr2ast(expression)

    # Create SymPy expression from the AST
    sympy_expr = ast2sympy(ast_tree)

    # print("SymPy expression:", sympy_expr)

    # Convert the SymPy expression back to AST
    new_ast_tree = sympy2ast(sympy_expr)
    # print('\n'+'*'*50+'\n')
    # print("Reconstructed AST:", ast.dump(new_ast_tree, indent=4))
    # print('\n'+'*'*50+'\n')

    # Use astor.to_source to convert the AST back to a string
    expression_str = astor.to_source(new_ast_tree)

    print("Reconstructed Expression String:", expression_str.strip())

    return new_ast_tree

# Example usage
expression = "c1 * (c2 * c2 * (r1 + c3) * (r1 + c3) - 1) * atan2(-0.5 * (r1 * r1 * c2 * c2), c1)"
new_ast = around(expression)

In [ ]:
from xmm.codegen.codegen import generate_expr
# Example usage
expression = "c1 * (c2 * c2 * (r1 + c3) * (r1 + c3) - 1) * atan2(-0.5 * (r1 * r1 * c2 * c2), c1)"
taichi_expr, taichi_derivatives = generate_expr(expression)

print('\n'+'*'*50+'\n')

print("Original CUDA Expression:\n", taichi_expr)
print("CUDA Derivatives:")
for var, expr in taichi_derivatives.items():
    print(f"d/d{var}:", expr)

In [ ]:
from xmm.codegen.codegen import generate_operator_source

a, b = generate_operator_source(1, 3, "c1 * (c2 * c2 * (r1 + c3) * (r1 + c3) - 1) * atan2(-0.5 * (r1 * r1 * c2 * c2), c1)")
print(b)

## Example Main

In [ ]:
from xmm.SumOperator import SumOperator
import torch

op = SumOperator(1, 3, "c1 * (c2 * c2 * (r1 + c3) * (r1 + c3) - 1) * atan2(-0.5 * (r1 * r1 * c2 * c1),c1 * c2)")
op.compile(identifier='testop_h934nvewq230g9')


In [130]:
import torch
device = torch.device('cuda:0')

M, N, K = 128, 42, 62
r1 = torch.randn(M, K).to(device)
c1 = torch.randn(N, K).to(device)
c2 = torch.randn(N, K).to(device)
c3 = torch.randn(N, K).to(device)


index = 15
opf = op.forward(r1,c1,c2,c3)
print(opf[index, index])
# import wavkan.wavelet
# print(wavkan.wavelet.mexhat_wavelet(r1,c1,c2,c3))
from math import cos, atan2
res = 0
for k in range(K):
    res += c1[index,k] * (c2[index,k] * c2[index,k] * (r1[index,k] + c3[index,k]) * (r1[index,k] + c3[index,k]) - 1) * \
    atan2(-0.5 * (r1[index,k] * r1[index,k] * c2[index,k] * c1[index,k]),c2[index,k] * c1[index,k])
print(res)
print(abs(res-opf[index, index]))

tensor(-77.4010, device='cuda:0')
tensor(-77.4010, device='cuda:0')
tensor(7.6294e-06, device='cuda:0')
